# model

In [1]:
import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")

In [2]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x, rat):
        x = x + (self.pe[:x.size(0), :]) * rat
        return self.dropout(x)

class TFModule(nn.Module):
    def __init__(self, vocab_size, embed_size, num_layers, num_hiddens, num_heads, embeddropout_rate, dropout_rate, pos_ratio):
        super(TFModule, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.pos_embed = PositionalEncoding(embed_size, embeddropout_rate, 1025)
        self.pos_ratio = pos_ratio
        
        self.encoder_layer = nn.TransformerEncoderLayer(
                                        d_model=embed_size,
                                        dim_feedforward=num_hiddens,
                                        nhead=num_heads, 
                                        dropout=dropout_rate, 
                                        activation='relu')
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        
        self.decoder = nn.Linear(embed_size, vocab_size) # 初始时间步和最终时间步的隐藏状态作为全连接层输入

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask.to(device)
    
    def forward(self, inputs):
        
        word_emb = self.embedding(inputs.permute(1, 0))
        embeddings = self.pos_embed(word_emb, self.pos_ratio)
#         print(word_emb, '\n', embeddings-word_emb)
        
        # outputs形状是(长度, 批量大小, embeding大小)
        outputs = self.transformer_encoder(embeddings, mask=self._generate_square_subsequent_mask(embeddings.shape[0]))

        output = outputs.permute(1, 0, 2)
        outs = output.reshape(output.size(0)*output.size(1), output.size(2))
        ret = self.decoder(outs)
        return ret

In [3]:
# import torch
# import torch.nn as nn

In [4]:
# encoder_layer = nn.TransformerEncoderLayer(d_model=1, nhead=1, dropout=0, activation='relu')
# transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)

In [5]:
# src = torch.ones(3,1,1)
# out = encoder_layer(src)
# print(src.shape,out.shape)
# print(src)
# print(out)

In [6]:
# # dir(encoder_layer)
# vos = 53
# emb = 22
# numl = 3
# numh = 233
# numhea = 11
# dror = 0
# model = TFModule(vos, emb, numl, numh, numhea, dror)
# src = torch.ones((2,3),dtype=int)
# print(src.shape)
# out = model(src)
# print(out)

# data

In [7]:
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

In [8]:
class Corpus(object):
    def __init__(self, train_batch_size=20, eval_batch_size=10, bptt=35):
        self.bptt = bptt
        train_iter = WikiText2(split='train')
        self.tokenizer = get_tokenizer('basic_english')
        counter = Counter()
        for line in train_iter:
            counter.update(self.tokenizer(line))
        self.vocab = Vocab(counter)
        train_iter, val_iter, test_iter = WikiText2()
        train_data = self.data_process(train_iter)
        val_data = self.data_process(val_iter)
        test_data = self.data_process(test_iter)

        self.train_data = self.batchify(train_data, train_batch_size)
        self.val_data = self.batchify(val_data, eval_batch_size)
        self.test_data = self.batchify(test_data, eval_batch_size)

    def data_process(self, raw_text_iter):
        data = [torch.tensor([self.vocab[token] for token in self.tokenizer(item)],
                           dtype=torch.long) for item in raw_text_iter]
        return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

    def batchify(self, data, batch_size):
        # Divide the dataset into batch_size parts.
        nbatch = data.size(0) // batch_size
        # Trim off any extra elements that wouldn't cleanly fit (remainders).
        data = data.narrow(0, 0, nbatch * batch_size)
        # Evenly divide the data across the batch_size batches.
        data = data.view(batch_size, -1).t().contiguous()
        return data.to(device)

    def get_batch(self, source, i):
        seq_len = min(self.bptt, len(source) - 1 - i)
        data = source[i:i+seq_len]
        target = source[i+1:i+1+seq_len].reshape(-1)
        return data, target

    def get_ntokens(self):
        return len(self.vocab.stoi)

# main

In [9]:
import argparse
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable

import data
import model
class args():
    pos_ratio = 1
    emsize = 100
    nhid = 200
    nlayers = 3
    nhead = 20
    lr = 0.5
    epochs = 2000
    batch_size = 32
    bptt = 64
    embeddropout = 0.1
    dropout = 0.3
    tied = False
    seed = 1234
    clip = 0.005
    save = 'model.pt'

In [10]:
data_loader = Corpus(train_batch_size=args.batch_size,
                     eval_batch_size=args.batch_size,
                     bptt=args.bptt)

In [11]:
train_data = data_loader.train_data
val_data = data_loader.val_data
test_data = data_loader.test_data

In [12]:
def get_batch(source, i, evaluation=False):
    seq_len = min(args.bptt, len(source) - 1 - i)
#     data = Variable(source[i:i+seq_len], volatile=evaluation)
#     target = Variable(source[i+1:i+1+seq_len].view(-1))
    data = source[i:i+seq_len].t()
    tmp = source[i+1:i+1+seq_len].t()
#     print(data.shape,tmp.shape)
    target = tmp.reshape(-1)
    return data, target

In [13]:
# import torch
# torch.__version__
# print(train_data.size(0))
# for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
#     data, targets = get_batch(train_data, i)
#     print(data.shape,targets.shape,targets[-1])
#     if (batch>=2): break

In [14]:
len(data_loader.vocab.itos)

28783

In [15]:
ntokens = len(data_loader.vocab.itos)
# model = RNNModel(args.model, ntokens, args.emsize, args.nhid,
#         args.nlayers, args.dropout, args.tied)
model = TFModule(ntokens, args.emsize, args.nlayers, args.nhid, args.nhead, args.embeddropout, args.dropout, args.pos_ratio)

In [16]:
model = model.to(device)

In [17]:
model

TFModule(
  (embedding): Embedding(28783, 100)
  (pos_embed): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=100, out_features=100, bias=True)
    )
    (linear1): Linear(in_features=100, out_features=200, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (linear2): Linear(in_features=200, out_features=100, bias=True)
    (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_fe

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)

In [19]:
def clip_gradient(optimizer, grad_clip):
    for group in optimizer.param_groups:
        for param in group["params"]:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)

def evaluate(data_source):
    model.eval()
    total_loss = 0
    total_words = 0
    for batch, i in enumerate(range(0, data_source.size(0) - 1, args.bptt)):
        data, targets = get_batch(data_source, i)
        
        output = model(data)
        loss = criterion(output, targets)

        total_loss += loss.data * targets.shape[0]
        total_words += targets.shape[0]
        
#         if batch > 1: break
        
    total_loss = float(total_loss.to(torch.device('cpu')))
#     print(total_loss,total_words)
    return total_loss / total_words, math.exp(total_loss / total_words)

def train():
    model.train()
    total_loss = 0.
    total_words = 0
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
        data, targets = get_batch(train_data, i)
        model.zero_grad()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        clip_gradient(optimizer,args.clip)
        optimizer.step()

        total_loss += loss.data * targets.shape[0]
        total_words += targets.shape[0]
        
#         if batch > 10: break
        
#         if (batch % args.log_interval == 0 and batch > 0):
#             cur_loss = total_loss / args.log_interval
#             elapsed = time.time() - start_time
#             print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
#                     'loss {:5.2f} | ppl {:8.2f}'.format(
#                 233, batch, len(train_data) // args.bptt, args.lr,
#                 elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
#             total_loss = 0
#             start_time = time.time()

    cur_loss = total_loss / total_words
    print('train {:10d} words | loss {:5.5f} | ppl {:5.5}'.format(total_words, cur_loss, math.exp(cur_loss)))
    return cur_loss, math.exp(cur_loss)

In [20]:
import numpy as np
Losstrain = []
Lossval = []
Losstest = []

In [ ]:
import time
for T in range(args.epochs):
    print('Round : ',T,"  ",time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
#     train()
#     print('  valid ',evaluate(val_data))
#     print('  test  ',evaluate(test_data))
    trainl, trainp = train()
    vall, valp = evaluate(val_data)
    testl, testp = evaluate(test_data)
    
    Losstrain.append(trainp)
    Lossval.append(valp)
    Losstest.append(testp)
    np.save('model-2-try2-train',np.array(Losstrain))
    np.save('model-2-try2-val',np.array(Lossval))
    np.save('model-2-try2-test',np.array(Losstest))
    
    print('  valid {:5.5f} | test {:5.5f}'.format(valp, testp))
    print('-'*80)

Round :  0    2021-05-21 10:23:26
train    2049952 words | loss 6.84823 | ppl 942.21
  valid 551.15593 | test 516.56571
--------------------------------------------------------------------------------
Round :  1    2021-05-21 10:24:45
train    2049952 words | loss 6.42837 | ppl 619.17
  valid 456.10070 | test 424.63366
--------------------------------------------------------------------------------
Round :  2    2021-05-21 10:26:04
train    2049952 words | loss 6.28390 | ppl 535.87
  valid 409.29662 | test 378.34212
--------------------------------------------------------------------------------
Round :  3    2021-05-21 10:27:22
train    2049952 words | loss 6.18977 | ppl 487.73
  valid 378.91049 | test 348.56095
--------------------------------------------------------------------------------
Round :  4    2021-05-21 10:28:40
train    2049952 words | loss 6.11680 | ppl 453.41
  valid 356.65362 | test 326.85929
----------------------------------------------------------------------------

train    2049952 words | loss 5.28866 | ppl 198.08
  valid 251.35190 | test 225.68683
--------------------------------------------------------------------------------
Round :  42    2021-05-21 11:18:32
train    2049952 words | loss 5.27966 | ppl 196.3
  valid 252.43397 | test 226.55949
--------------------------------------------------------------------------------
Round :  43    2021-05-21 11:19:50
train    2049952 words | loss 5.27117 | ppl 194.64
  valid 252.02362 | test 226.33100
--------------------------------------------------------------------------------
Round :  44    2021-05-21 11:21:08
train    2049952 words | loss 5.26250 | ppl 192.96
  valid 252.57723 | test 226.44112
--------------------------------------------------------------------------------
Round :  45    2021-05-21 11:22:27
train    2049952 words | loss 5.25413 | ppl 191.35
  valid 252.19739 | test 225.32722
--------------------------------------------------------------------------------
Round :  46    2021-05-21 

train    2049952 words | loss 5.07555 | ppl 160.06
  valid 257.55196 | test 232.87931
--------------------------------------------------------------------------------
Round :  83    2021-05-21 12:12:19
train    2049952 words | loss 5.07293 | ppl 159.64
  valid 257.49190 | test 231.79089
--------------------------------------------------------------------------------
Round :  84    2021-05-21 12:13:39
train    2049952 words | loss 5.07186 | ppl 159.47
  valid 259.27366 | test 234.31317
--------------------------------------------------------------------------------
Round :  85    2021-05-21 12:14:58
train    2049952 words | loss 5.06593 | ppl 158.53
  valid 258.63675 | test 233.24059
--------------------------------------------------------------------------------
Round :  87    2021-05-21 12:17:36
train    2049952 words | loss 5.06391 | ppl 158.21
  valid 258.20728 | test 232.19825
--------------------------------------------------------------------------------
Round :  88    2021-05-21

train    2049952 words | loss 4.99777 | ppl 148.08
  valid 257.70925 | test 232.36911
--------------------------------------------------------------------------------
Round :  125    2021-05-21 13:07:31
train    2049952 words | loss 4.99636 | ppl 147.87
  valid 259.75290 | test 234.50001
--------------------------------------------------------------------------------
Round :  126    2021-05-21 13:08:50
train    2049952 words | loss 4.99595 | ppl 147.81
  valid 258.85174 | test 235.57533
--------------------------------------------------------------------------------
Round :  127    2021-05-21 13:10:09
train    2049952 words | loss 4.99446 | ppl 147.59
  valid 258.05330 | test 233.66379
--------------------------------------------------------------------------------
Round :  128    2021-05-21 13:11:27
train    2049952 words | loss 4.99309 | ppl 147.39
  valid 259.48404 | test 235.78841
--------------------------------------------------------------------------------
Round :  129    2021-

train    2049952 words | loss 4.96581 | ppl 143.43
  valid 254.58420 | test 230.55913
--------------------------------------------------------------------------------
Round :  166    2021-05-21 14:01:18
train    2049952 words | loss 4.96496 | ppl 143.3
  valid 255.22214 | test 233.60015
--------------------------------------------------------------------------------
Round :  167    2021-05-21 14:02:36
train    2049952 words | loss 4.96498 | ppl 143.31
  valid 255.92346 | test 231.02164
--------------------------------------------------------------------------------
Round :  168    2021-05-21 14:03:55
train    2049952 words | loss 4.96575 | ppl 143.42
  valid 255.83707 | test 233.11211
--------------------------------------------------------------------------------
Round :  169    2021-05-21 14:05:14
train    2049952 words | loss 4.96459 | ppl 143.25
  valid 253.29432 | test 228.99916
--------------------------------------------------------------------------------
Round :  170    2021-0

...

train    2049952 words | loss 4.71542 | ppl 111.66
  valid 246.76456 | test 223.94487
--------------------------------------------------------------------------------
Round :  946    2021-05-22 07:05:10
train    2049952 words | loss 4.71651 | ppl 111.78
  valid 256.86415 | test 240.63027
--------------------------------------------------------------------------------
Round :  947    2021-05-22 07:06:29
train    2049952 words | loss 4.71493 | ppl 111.6
  valid 244.45377 | test 222.10727
--------------------------------------------------------------------------------
Round :  948    2021-05-22 07:07:48
train    2049952 words | loss 4.71516 | ppl 111.63
  valid 255.23703 | test 239.95842
--------------------------------------------------------------------------------
Round :  949    2021-05-22 07:09:06
train    2049952 words | loss 4.71481 | ppl 111.59
  valid 249.03126 | test 225.04693
--------------------------------------------------------------------------------
Round :  950    2021-0

train    2049952 words | loss 4.72004 | ppl 112.17
  valid 257.57810 | test 241.30125
--------------------------------------------------------------------------------
Round :  987    2021-05-22 07:58:55
train    2049952 words | loss 4.71876 | ppl 112.03
  valid 245.39064 | test 223.20455
--------------------------------------------------------------------------------
Round :  988    2021-05-22 08:00:13
train    2049952 words | loss 4.72053 | ppl 112.23
  valid 256.95464 | test 241.47082
--------------------------------------------------------------------------------
Round :  989    2021-05-22 08:01:32
train    2049952 words | loss 4.71917 | ppl 112.07
  valid 244.28178 | test 222.07857
--------------------------------------------------------------------------------
Round :  990    2021-05-22 08:02:52
train    2049952 words | loss 4.72169 | ppl 112.36
  valid 258.49984 | test 243.28873
--------------------------------------------------------------------------------
Round :  991    2021-

  valid 268.42907 | test 252.33026
--------------------------------------------------------------------------------
Round :  1027    2021-05-22 08:51:23
train    2049952 words | loss 4.74230 | ppl 114.7
  valid 252.68153 | test 228.08527
--------------------------------------------------------------------------------
Round :  1028    2021-05-22 08:52:42
train    2049952 words | loss 4.74492 | ppl 115.0
  valid 263.15461 | test 246.88810
--------------------------------------------------------------------------------
Round :  1029    2021-05-22 08:54:01
train    2049952 words | loss 4.74450 | ppl 114.95
  valid 250.06149 | test 224.94156
--------------------------------------------------------------------------------
Round :  1030    2021-05-22 08:55:18
train    2049952 words | loss 4.74640 | ppl 115.17
  valid 263.10367 | test 247.21004
--------------------------------------------------------------------------------
Round :  1031    2021-05-22 08:56:37
train    2049952 words | loss 4.7

train    2049952 words | loss 4.77267 | ppl 118.23
  valid 252.08859 | test 226.97314
--------------------------------------------------------------------------------
Round :  1068    2021-05-22 09:45:07
train    2049952 words | loss 4.77534 | ppl 118.55
  valid 269.95007 | test 253.40101
--------------------------------------------------------------------------------
Round :  1069    2021-05-22 09:46:26
train    2049952 words | loss 4.77327 | ppl 118.31
  valid 250.79776 | test 225.72475
--------------------------------------------------------------------------------
Round :  1070    2021-05-22 09:47:44
train    2049952 words | loss 4.77584 | ppl 118.61
  valid 270.40206 | test 254.59762
--------------------------------------------------------------------------------
Round :  1071    2021-05-22 09:49:03
train    2049952 words | loss 4.77385 | ppl 118.37
  valid 250.57907 | test 225.92599
--------------------------------------------------------------------------------
Round :  1072    

  valid 253.41842 | test 229.51631
--------------------------------------------------------------------------------
Round :  1108    2021-05-22 10:37:34
train    2049952 words | loss 4.81450 | ppl 123.28
  valid 267.62761 | test 250.78886
--------------------------------------------------------------------------------
Round :  1109    2021-05-22 10:38:53
train    2049952 words | loss 4.81754 | ppl 123.66
  valid 266.23439 | test 239.06443
--------------------------------------------------------------------------------
Round :  1110    2021-05-22 10:40:12
train    2049952 words | loss 4.82205 | ppl 124.22
  valid 267.69956 | test 250.76968
--------------------------------------------------------------------------------
Round :  1111    2021-05-22 10:41:31
train    2049952 words | loss 4.81693 | ppl 123.58
  valid 255.86870 | test 230.21687
--------------------------------------------------------------------------------
Round :  1112    2021-05-22 10:42:50
train    2049952 words | loss 4

In [ ]:
# torch.save(model.state_dict(), 'parameter-2.pkl')
# evaluate(test_data)

In [ ]:
# if 1:
#     x = torch.ones((1,1),dtype=int)
#     x[0,0] = 1
#     print(model.embedding(x))

In [ ]:
# import time
# for T in range(100):
#     print('Round : ',T,"  ",time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
#     train()
#     print('  valid ',evaluate(val_data))
#     print('  test  ',evaluate(test_data))
#     print('-'*80)

In [ ]:
# if 1:
#     x = torch.tensor([[1000,1],[100,1],[1000,1]],dtype=float)
#     y = torch.tensor([0,1,0])
#     print(criterion(x,y))

In [ ]:
# -math.log(0.1)